## Training Amazon SageMaker models for binding affinity prediction by using the DGL-LifeSci with PyTorch backend
The **Amazon SageMaker Python SDK** makes it easy to train DGL-LifeSci models. In this example, you train an Atomic Convolutional Networks (ACNN) [1] or PotentialNet [2] model using the [PDBBind](http://www.pdbbind.org.cn/) dataset. For detailed information of them please see [the example page of DGL-Lifesci](https://github.com/yoheigon/dgl-lifesci/tree/master/examples/binding_affinity_prediction).

[1] Gomes et al. (2017) Atomic Convolutional Networks for Predicting Protein-Ligand Binding Affinity. *arXiv preprint arXiv:1703.10603*.

[2] Feinberg et al. (2018) PotentialNet for molecular property prediction. *ACS central science* 4.11: 1520-1530.

### Setup
Define a few variables that are needed later in the example.

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session

# Setup session
sess = sagemaker.Session()

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here.
bucket = sess.default_bucket()

# IAM execution role that gives Amazon SageMaker access to resources in your AWS account.
# You can use the Amazon SageMaker Python SDK to get the role from the notebook environment.
role = get_execution_role()

### The training script
The main.py script provides all the code you need for training an Amazon SageMaker model. 

In [ ]:
!cat ./code/main.py

### SageMaker's Estimator class
The Amazon SageMaker Estimator allows you to run single machine in Amazon SageMaker, using CPU or GPU-based instances.

When you create the estimator, pass in the filename of the training script and the name of the IAM execution role. You can also provide a few other parameters. instance_count and instance_type determine the number and type of Amazon SageMaker instances that are used for the training job. The hyperparameters parameter is a dictionary of values that is passed to your training script as parameters so that you can use argparse to parse them. You can see how to access these values in the main.py script above.

For this example, we use ml.p3.2xlarge for training instance, PDBBind(v2015) core for dataset, and PDBBind for model. 

In [ ]:
from sagemaker.pytorch import PyTorch

metric_definitions = [
    {"Name": "mae", "Regex": "mae ([0-9.]+).*$"},
    {"Name": "r2", "Regex": "r2 ([0-9.]+).*$"},
]

# Create estimator
estimator = PyTorch(
    entry_point="main.py",
    source_dir="code",
    role=role,
    framework_version="1.6.0",
    py_version="py3",
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    hyperparameters={
        "model": "PotentialNet",
        "dataset_option": "PDBBind_core_pocket_random",
        "version": "v2015",
    },
    metric_definitions=metric_definitions,
)

### Running the Training Job
After you construct the Estimator object, fit it by using Amazon SageMaker. The [PDBBind](http://www.pdbbind.org.cn/) dataset is automatically downloaded.

In [ ]:
# Launch SageMaker training job
estimator.fit()

### (Optional) Hyperparameter tuning

SageMaker offers hyperparameter tuning to kick off multiple training jobs with different hyperparameter combinations, to find the set with best model performance. You can specify the hyperparameters you want to tune and their possible values.

In [ ]:
import boto3
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.01),
    "num_epochs": IntegerParameter(100, 200),
}

Next, specify the objective metric that you want to tune and its definition. This includes the regular expression (regex) needed to extract that metric from the Amazon CloudWatch logs of the training job. For this example, "mae" and "r2" is supported.

In [ ]:
objective_metric_name = "mae"

Now, create a HyperparameterTuner object, which you pass:

 * The training estimator you created above
 * The hyperparameter ranges
 * Objective metric name and definition
 * Number of training jobs to run in total and how many training jobs should be run simultaneously. More parallel jobs will finish tuning sooner, but may sacrifice accuracy. We recommend you set the parallel jobs value to less than 10% of the total number of training jobs (we'll set it higher just for this example to keep it short).
 * Whether you should maximize or minimize the objective metric. You haven't specified here since it defaults to 'Maximize', which is what you want for validation accuracy
 

In [ ]:
task_tags = [{"Key": "ML Task", "Value": "DGL-Lifesci"}]
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    tags=task_tags,
    max_jobs=2,
    max_parallel_jobs=1,
)

And finally, start the tuning job by calling .fit().

In [ ]:
tuner.fit(wait=False)

Let's just run a quick check of the hyperparameter tuning jobs status to make sure it started successfully and is InProgress.

In [ ]:
sagemaker_client = boto3.client("sagemaker")

sagemaker_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)["HyperParameterTuningJobStatus"]

### Output
After the hyperparameter tuning job is finished. You can get the best model of the turning job as below. Through the SageMaker hyperparameter turning job, you can easily manage computationally heavy training jobs and find the best-performing model configuration.

In [ ]:
s3_client = boto3.client("s3")

tuner_status = sagemaker_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)["HyperParameterTuningJobStatus"]

if tuner_status == "Completed":
    best_training_job_summary = sagemaker_boto_client.describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)["BestTrainingJob"]
    best_training_job_details = sagemaker_boto_client.describe_training_job(
        TrainingJobName=best_training_job_summary["TrainingJobName"])

    trained_model_s3_uri = best_training_job_details["ModelArtifacts"]["S3ModelArtifacts"]
    s3_client.download_file(trained_model_s3_uri.split('/')[2], ('/').join(trained_model_s3_uri.split('/')[3:]), "model.tar.gz")